In [1]:
import requests
import requests_mock
from sdxlib.client import SDXClient, SDXException

In [2]:
# Define a mock response dictionary for successful L2VPN creation
mock_response_successful = {
    "service_id": "12345",
}
mock_response_unsuccessful = {
    "description": "description of failure"
}

In [3]:
url = "https://example.com"
client_name = "Test L2VPN"
client_endpoints = [
    {"port_id": "urn:sdx:port:sax.br:Rtr01:50", "vlan": "any"},
    {"port_id": "urn:sdx:port:ampath.net:Ampath3:50", "vlan": "any"}
]

In [4]:
client = SDXClient(url, client_name, client_endpoints)

In [5]:
client

SDXClient(base_url=https://example.com, name=Test L2VPN, endpoints=[{'port_id': 'urn:sdx:port:sax.br:Rtr01:50', 'vlan': 'any'}, {'port_id': 'urn:sdx:port:ampath.net:Ampath3:50', 'vlan': 'any'}])

In [6]:
# Create a mocker for requests
with requests_mock.Mocker() as mock_requests:
    # Mock successful L2VPN creation
    mock_requests.register_uri(
        "POST", f"{url}/l2vpn/1.0", 
        json=mock_response_successful,
        status_code=201  # Set status code for successful creation
    )
    
    # Adding debug statement to confirm URL and method
    #print(f"Mocking URL: {url}/l2vpn/1.0")

    try:
        response = client.create_l2vpn()  # Assuming create_l2vpn doesn't take arguments
        print("L2VPN creation successful!")
        print(f"Response: {response}")  # Print the mocked response (service_id in this case)
    except SDXException as e:
        print(f"L2VPN creation failed: {e}")  # Handle potential exceptions


L2VPN creation request sent to https://example.com/l2vpn/1.0. It may take several seconds to receive a response.
L2VPN creation successful!
Response: {'service_id': '12345'}


In [ ]:
# Disable Mocks
mock_requests.clear()